## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

"wget" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [2]:
import pandas as pd
import numpy as np

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.


In [25]:
data = pd.read_csv('data_000637.csv')
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0



2\. Find out the number of BX in a ORBIT (the value 'x').


In [15]:
X = max(data['BX_COUNTER']) + 1
print(X)

3564



3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.


In [42]:
orbit_time = X * 25
max_orbit_row = data.loc[data['ORBIT_CNT'] == data['ORBIT_CNT'].max()]
min_orbit_row = data.loc[data['ORBIT_CNT'] == data['ORBIT_CNT'].min()]
max_bx_row = max_orbit_row.loc[max_orbit_row['BX_COUNTER'] == max_orbit_row['BX_COUNTER'].max()]
min_bx_row = min_orbit_row.loc[min_orbit_row['BX_COUNTER'] == min_orbit_row['BX_COUNTER'].min()]
max_tdc_row = max_bx_row.loc[max_bx_row['TDC_MEAS'] == max_bx_row['TDC_MEAS'].max()]
min_tdc_row = min_bx_row.loc[min_bx_row['TDC_MEAS'] == min_bx_row['TDC_MEAS'].min()]
print(max_tdc_row)
print('\n\n')
print(min_tdc_row)
print('\n\n')
max_tdc_row_time = max_tdc_row['ORBIT_CNT'] * orbit_time + max_tdc_row['BX_COUNTER'] * 25 + max_tdc_row['TDC_MEAS'] * (25 / 30)
min_tdc_row_time = min_tdc_row['ORBIT_CNT'] * orbit_time + min_tdc_row['BX_COUNTER'] * 25 + min_tdc_row['TDC_MEAS'] * (25 / 30)
#measurement_time = max_tdc_row_time - min_tdc_row_time
print(type(max_tdc_row_time))

         HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
1310718     1     0          139  3869211171         769         0



   HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0     1     0          123  3869200167        2374        26



<class 'pandas.core.series.Series'>



4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).



5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139